Purpose: Turn text-data into data with appropriate values neccessary for functional RAGAS evaluation

Fetch documents (from only the SciComp wiki as of now) and put into accessable format for generation of expected outputs and creation of context later on

In [1]:
# from langchain_community.document_loaders import DirectoryLoader
# # import nltk
# # nltk.download('punkt')
# loader = DirectoryLoader("../data/test")
# documents = loader.load()


from langchain_community.document_loaders import TextLoader
import os

loader = TextLoader("../data/test/test.txt")
documents = loader.load()
print(documents)



# #Code that indexes over all the documents in the Data folder below 
# import glob
# import os

# class TextLoader:
#     def __init__(self, base_path):
#         self.base_path = base_path

#     def load(self):
#         documents = []
#         # Use glob.glob with recursive=True to find all .txt files within the base_path directory and its subdirectories
#         #refactor to work with non txt files
#         for file_path in glob.glob(os.path.join(self.base_path, '**/*.txt'), recursive=True):
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 documents.append(file.read())
#         return documents

# # Assuming the base_path is set correctly relative to the notebook's location
# loader = TextLoader("../data")
# documents = loader.load()
# print(documents)

[Document(page_content='Overview of Janelia Scientific Computing\nThe Janelia Scientific Computing team supports the institute’s mission by developing and maintaining computational infrastructure, software tools, and data analysis pipelines essential for modern biological research. They collaborate closely with scientists to address computational challenges and drive innovation in various research areas, including neuroscience, cell biology, and bioinformatics.\n\nKey Areas of Focus\nHigh-Performance Computing (HPC):\n\nProvides access to powerful computing clusters to handle large-scale data analysis and complex simulations.\nOptimizes computational workflows to maximize efficiency and resource utilization.\nData Management and Storage:\n\nDevelops and maintains robust data storage solutions to handle the vast amounts of data generated by research activities.\nEnsures data integrity, security, and accessibility.\nSoftware Development:\n\nCreates custom software tools and platforms tai

Generate the testset

In [2]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
# generator with openai models
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file
api_key = os.environ.get("OPENAI_API_KEY")

#change the model to a better one once the whole thing is functional

generator_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)
# critic_llm = ChatOpenAI(model="gpt-4")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)
embeddings = OpenAIEmbeddings(api_key=api_key)

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
) 


testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

/Users/bakhalea/Documents/gpt-semantic-search/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filename and doc_id are the same for all nodes.               
Generating: 100%|██████████| 10/10 [01:00<00:00,  6.10s/it]


Address potentiall duplicate questions (which the testset generator has done) and delete their rows

In [4]:
df = testset.to_pandas()
df = df.drop_duplicates(subset='question', keep='first')
questions_list = df['question'].tolist()
print (questions_list)

seen = set()
questions_list = [x for x in questions_list if not (x in seen or seen.add(x))]

for item in questions_list:
    print(item)

    

# testset.to_json("testset.json")
# Creates dataset of ground truths contexts and questions for the testset
# Missing answers column 
# On one medium size document, it took about 1 minutes to generate 10 questions and cost 3 dollars on OpenAI
# Seems expensive when using gpt-4, is its use justified or does 3.5 get the job done?
# Evaluate gpt-4 vs gpt-3.5-turbo-16k for RAGAS evaluation test data generation


['How does the Janelia Scientific Computing team support advanced imaging techniques and image analysis in various research areas?', 'How does the Janelia Scientific Computing team support advanced imaging techniques and image analysis in research areas such as neuroscience and cell biology?', 'How does the Janelia Scientific Computing team utilize high-performance computing to handle large-scale data analysis and complex simulations in modern biological research?', 'What has the Janelia Scientific Computing team done in bioinformatics and data analysis, particularly with the FlyEM Project and Neuroglancer, and how does it affect neuroscience and cell biology research?', 'How does the Janelia Scientific Computing team support advanced imaging and analysis in neuroscience, cell biology, and bioinformatics through collaborations and custom computational solutions?']
How does the Janelia Scientific Computing team support advanced imaging techniques and image analysis in various research a

fetch the LLM's response and append to a list

In [5]:
# Assuming SemanticSearchParser is your class and it has a method named 'generate_response' that takes a question and returns an answer

# Initialize your SemanticSearchParser class
# Adjust this step if your class initialization requires different parameters


# Now you can import the class

from generate_answer import SemanticSearchService

weaviate_url = "http://localhost:8777"
service = SemanticSearchService(weaviate_url)
print (questions_list)

# List to store answers (optional)
answers_list = []


# Loop through each question in the questions_list
for question in questions_list:
    # Use the question as input to get the answer
    answer = service.generate_response(question)
    
    # Print the answer
    
    # Optionally, append the answer to answers_list for further processing
    answers_list.append(answer)

print (answers_list)
temp_ans_list = ["The Janelia Scientific Computing team operates and maintains a world-class computational infrastructure that includes a high-performance compute cluster with over 5000 cores and 300 GPUs. This infrastructure is used to analyze and mine the large amounts of data produced by Janelia's scientists. The team also supports a state-of-the-art storage and compute infrastructure across two data centers, which currently supports over 15 petabytes of scientific data. This data is split across various storage tiers and is connected with an optical fiber ring. The team also maintains a 4500 sq ft data center with significant power and cooling capacity. \n\nIn addition to hardware, the team also has deep software skills in a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. These skills are used to help with research and engineering tasks, from quick questions to full software life cycle support. The team's software skills, combined with their domain knowledge in areas such as image processing, machine learning, data handling, microscopy, instrument control, 3D graphics & visualization, and bioinformatics & transcriptomics, allow them to efficiently work with both experimentalists and computer scientists. \n\nThe team also develops and maintains a variety of tools and projects, such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher, which are used for various aspects of data analysis and simulation in biological research.", "The Janelia Scientific Computing team provides a wide range of support for advanced imaging techniques and image analysis. They offer consultation on experiment design as well as image visualization and processing. They also provide comprehensive image and data analysis support for multiple software packages through hands-on assistance and/or custom-written macros/plugins/scripts for ImageJ/FIJI, MATLAB, Imaris, etc. \n\nIn addition, they maintain several computer workstations dedicated to viewing and processing large image datasets acquired with the facility's instruments. These workstations are equipped with a suite of imaging software, including a full version of Imaris, and have robust hardware specifications to handle large datasets. \n\nThe team also has deep domain knowledge in image processing, machine learning, data handling, and 3D graphics & visualization, which allows them to efficiently work with experimentalists and computer scientists in various research areas.", "The Janelia Scientific Computing team provides world-class computational infrastructure to support the institute's scientific endeavors. They operate and maintain all of Janelia’s storage and associated backup infrastructure, high performance compute cluster, and all Linux systems. They also manage Janelia’s data center and backup and disaster recovery resources. The team supports a Linux compute cluster with over 5000 cores and 300 GPUs, and is responsible for maintaining many other Linux servers and workstations. They also handle a significant amount of data, with almost 100TB of new Janelia’s data being safely backed up every month.\n\nIn addition to infrastructure, the Scientific Computing Software team works closely with Janelia's labs and project teams, providing everything from answering quick questions to full software life cycle support. They have a broad range of software skills, including programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They also have deep domain knowledge in areas like image processing, machine learning, data handling, microscopy, instrument control, 3D graphics & visualization, and bioinformatics & transcriptomics. \n\nThe team also identifies opportunities for code reuse, reducing development overhead and support costs across Janelia. They are strong proponents of open science and have created the Open Science Software Initiative. Most of their software is open source and available via GitHub. They also run a Scientific Computing Associates program to embed associates in SciComp and the lab or team they work with. \n\nThe team is led by Stephan Preibisch and consists of three teams: Software Engineering headed by Konrad Rokicki, Computational Methods and Solutions, both headed by Stephan Preibisch. They have developed several tools and projects like NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher. \n\nIn summary, the Janelia Scientific Computing team supports the institute's mission and drives innovation in modern biological research by providing robust computational infrastructure, software support, and developing innovative tools and solutions.", 'The Janelia Scientific Computing team supports advanced imaging techniques in neuroscience and cell biology through a variety of ways. They have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. They also develop and maintain a range of software tools and applications that aid in these areas. Some of these tools include NeuronBridge for finding neuron matches across modalities, HortaCloud for cloud-based collaborative annotation, VVD Viewer for volumetric rendering of 3D/4D microscopy data, and BigStitcher for efficient alignment of multi-tile and multi-angle image datasets. They also work closely with labs and project teams, providing full software life cycle support.', "The Janelia Scientific Computing team supports modern biological research in several ways. They maintain a world-class computational infrastructure, including storage and backup infrastructure, a high-performance compute cluster, and all Linux systems. They also manage Janelia's data center and backup and disaster recovery resources. The team supports data storage infrastructure for storing and accessing scientific data, with over 15 petabytes of scientific data split across various storage tiers. They also support a Linux compute cluster with over 5000 cores and 300 GPUs, and maintain many other Linux servers and workstations. \n\nIn addition to infrastructure support, the Scientific Computing Software team works closely with Janelia's labs, project teams, and shared resources to help with research and engineering tasks. They provide everything from answering quick questions to full software life cycle support. The team's software skills span a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They also have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. \n\nThe team also develops and maintains a variety of tools and projects, such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher. They are strong proponents of open science and most of their software is open source and available via GitHub. They also run the Scientific Computing Associates program, which offers challenging assignments for those interested in computational science.", "The Janelia Scientific Computing team provides comprehensive support for advanced imaging techniques in neuroscience, cell biology, and bioinformatics through a variety of collaborations and custom software tools. They work closely with Janelia's labs, project teams, and shared resources to assist with research and engineering tasks. This can range from answering quick questions to providing full software life cycle support.\n\nThe team's software skills cover a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. Many team members have backgrounds in biology, enabling them to work efficiently with experimentalists and computer scientists.\n\nThe team is also involved in various projects and tools such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher, which are designed to support advanced imaging techniques and data analysis in neuroscience, cell biology, and bioinformatics.\n\nFurthermore, the team is a strong proponent of open science and has teamed up with the Computation & Theory research area to create the Open Science Software Initiative. Most of their software is open source and available via GitHub, promoting collaboration and knowledge sharing. They also run the Scientific Computing Associates program, which embeds associates in SciComp and the lab or team they work with."]




['How does the Janelia Scientific Computing team support advanced imaging techniques and image analysis in various research areas?', 'How does the Janelia Scientific Computing team support advanced imaging techniques and image analysis in research areas such as neuroscience and cell biology?', 'How does the Janelia Scientific Computing team utilize high-performance computing to handle large-scale data analysis and complex simulations in modern biological research?', 'What has the Janelia Scientific Computing team done in bioinformatics and data analysis, particularly with the FlyEM Project and Neuroglancer, and how does it affect neuroscience and cell biology research?', 'How does the Janelia Scientific Computing team support advanced imaging and analysis in neuroscience, cell biology, and bioinformatics through collaborations and custom computational solutions?']
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function

Add values from the list of JaneliaGPT responses to the dataframe

In [8]:
df['answer'] = None
# Assuming df is your DataFrame and answers_list is a list with values to populate the 'Answer' column
if len(df) == len(answers_list):
    df['Answer'] = answers_list
else:
    print("The length of answers_list does not match the number of rows in the DataFrame.")

df.dropna(axis=1, how='all', inplace=True)

In [10]:


display(df)
df.to_json('WithAnswersDatasetFromTestTxt.json', index=True)

,question,contexts,ground_truth,evolution_type,metadata,episode_done,Answer
0,How does the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team supports...,simple,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team supports...
2,How does the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team supports...,simple,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team supports...
4,How does the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team utilizes...,simple,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team utilizes...
5,What has the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team has prov...,reasoning,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team has been...
6,How does the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team supports...,reasoning,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team supports...


In [102]:
##Attempt to fix the issue with the answers_list not bsjdkflasdjjfklasdkl fasdjkfh jkas
#df_fix = df
## List of columns to keep
#columns_to_keep = ['question', 'ground_truth', 'answer', 'contexts']
#
## Reassign df to a DataFrame containing only the columns to keep
#df_fix= df_fix[columns_to_keep]
#
#display(df_fix['contexts'])

0     [Overview of Janelia Scientific Computing\nThe Janelia Scientific Computing team supports the institute’s mission by developing and maintaining computational infrastructure, software tools, and data analysis pipelines essential for modern biological research. They collaborate closely with scientists to address computational challenges and drive innovation in various research areas, including neuroscience, cell biology, and bioinformatics.\n\nKey Areas of Focus\nHigh-Performance Computing (HPC):\n\nProvides access to powerful computing clusters to handle large-scale data analysis and complex simulations.\nOptimizes computational workflows to maximize efficiency and resource utilization.\nData Management and Storage:\n\nDevelops and maintains robust data storage solutions to handle the vast amounts of data generated by research activities.\nEnsures data integrity, security, and accessibility.\nSoftware Development:\n\nCreates custom software tools and platforms tailored to specific

In [104]:
# import ast

# # Function to safely convert string representation of a list to a list
# def convert_to_list(value):
#     try:
#         # Safely evaluate the string as a list
#         return ast.literal_eval(value)
#     except ValueError:
#         # Return an empty list in case of error
#         return []

# # Ensure 'contexts' is a list of lists
# df_fix['contexts'] = df_fix['contexts'].apply(lambda x: convert_to_list(x) if isinstance(x, str) else x)

# # Ensure each element in 'contexts' is a list of strings (if not already)
# df_fix['contexts'] = df_fix['contexts'].apply(lambda x: [x] if isinstance(x, str) else x)
#display (df_fix['contexts'])

0     [Overview of Janelia Scientific Computing\nThe Janelia Scientific Computing team supports the institute’s mission by developing and maintaining computational infrastructure, software tools, and data analysis pipelines essential for modern biological research. They collaborate closely with scientists to address computational challenges and drive innovation in various research areas, including neuroscience, cell biology, and bioinformatics.\n\nKey Areas of Focus\nHigh-Performance Computing (HPC):\n\nProvides access to powerful computing clusters to handle large-scale data analysis and complex simulations.\nOptimizes computational workflows to maximize efficiency and resource utilization.\nData Management and Storage:\n\nDevelops and maintains robust data storage solutions to handle the vast amounts of data generated by research activities.\nEnsures data integrity, security, and accessibility.\nSoftware Development:\n\nCreates custom software tools and platforms tailored to specific

Run the evaluation given the dataframe (does not work)

In [105]:



from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)


from ragas import evaluate


# Will reutrn a dataframe with the metrics
# Returns error for now because answers column is missing
# Error is misleading, fix dataset first and make it match the docs example dataset

result = evaluate(
    df,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

result.to_pandas()


AttributeError: 'DataFrame' object has no attribute 'rename_columns'